<a href="https://colab.research.google.com/github/agarwalsourabh55/huggingface-tapas/blob/sourabh_dev/huggingface_tapas_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers torch-scatter

!pip install transformers
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+${CUDA}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.9.0+.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=3567064 sha256=9d67f140dfa659898d5125c7283c689ed97b7462e1c23b64decf0e4f6dd52e76
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


### Different Types of Declaring TAPAS models


1.   Default SQA Configutaion
2.   with WTQ Configuration
3.   With WikiSQL Configuration


In [ ]:
from transformers import TapasConfig, TapasForQuestionAnswering

# for example, the base sized model with default SQA configuration
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")

# or, the base sized model with WTQ configuration
config = TapasConfig.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

# # or, the base sized model with WikiSQL configuration
# config = TapasConfig( "google/tapas-base-finetuned-wikisql-supervised",
#                         num_aggregation_labels=3,
#                         average_logits_per_cell=True)
# model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'column_output_bias', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'aggregation_classifier.weight', 'output_bias', 'aggregation_classifier.bias', 'column_output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



### Defining the Tokenizer class


1.   Defining the data labels.
2.   Defining the querie.
3.   Defining the answers of that queries.
4.   Converting all into the DAtaframe if not.
5.   Tokenizer Takes Text Only. 
6.   Advised to use Dataloaders for training the TAPAS



In [ ]:
from transformers import TapasTokenizer
import pandas as pd

model_name = "google/tapas-base"
tokenizer = TapasTokenizer.from_pretrained(model_name)

data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
queries = [
    "What is the name of the first actor?",
    "How many movies has George Clooney played in?",
    "What is the total number of movies?",
]
answer_coordinates = [[(0, 0)], [(2, 1)], [(0, 1), (1, 1), (2, 1)]]
answer_text = [["Brad Pitt"], ["69"], ["209"]]
table = pd.DataFrame.from_dict(data)
inputs = tokenizer(
    table=table,
    queries=queries,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)
inputs

{'input_ids': tensor([[ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2116,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0]]), 'labels': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'numeric_values': tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]]), 'numeric_values_scale': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [ ]:
table

,Actors,Number of movies
0,Brad Pitt,87
1,Leonardo Di Caprio,53
2,George Clooney,69


## TAPAS for Question Answering pretrained

Using Different methodology to defint the predefine the logits

In [ ]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

data = {
    "Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"],
    "Age": ["56", "45", "59"],
    "Number of movies": ["87", "53", "69"],
}
table = pd.DataFrame.from_dict(data)
queries = ["How many movies has George Clooney played in?", "How old is Brad Pitt?"]

inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
outputs = model(**inputs)

logits = outputs.logits
logits_aggregation = outputs.logits_aggregation

### Taking the Inference of TAPAS Pretrained models using the cricketers data.
Conducted a small POC to check how accurately TAPAS model used the predefined modes and predict the modelling approach

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
data = pd.read_csv("/content/gdrive/MyDrive/graduate_rate.csv")
data = data.astype(str)

Mounted at /content/gdrive


In [ ]:
data

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,School,601,0103,Appling County,Appling County High School,ALL Students,93.18
1,School,601,0103,Appling County,Appling County High School,American Indian/Alaskan,Too Few Students
2,School,601,0103,Appling County,Appling County High School,Asian/Pacific Islander,Too Few Students
3,School,601,0103,Appling County,Appling County High School,Black,84.21
4,School,601,0103,Appling County,Appling County High School,Economically Disadvantaged,93.18
...,...,...,...,...,...,...,...
6678,State,ALL,ALL,All Systems,All Schools,English Learners,68.49
6679,State,ALL,ALL,All Systems,All Schools,Hispanic,80.68
6680,State,ALL,ALL,All Systems,All Schools,Multi-Racial,87.08
6681,State,ALL,ALL,All Systems,All Schools,Students With Disability,74.01


In [ ]:
data.dropna()

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,School,601,0103,Appling County,Appling County High School,ALL Students,93.18
1,School,601,0103,Appling County,Appling County High School,American Indian/Alaskan,Too Few Students
2,School,601,0103,Appling County,Appling County High School,Asian/Pacific Islander,Too Few Students
3,School,601,0103,Appling County,Appling County High School,Black,84.21
4,School,601,0103,Appling County,Appling County High School,Economically Disadvantaged,93.18
...,...,...,...,...,...,...,...
6678,State,ALL,ALL,All Systems,All Schools,English Learners,68.49
6679,State,ALL,ALL,All Systems,All Schools,Hispanic,80.68
6680,State,ALL,ALL,All Systems,All Schools,Multi-Racial,87.08
6681,State,ALL,ALL,All Systems,All Schools,Students With Disability,74.01


,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
count,442,442,442,442,442,442,442
unique,1,188,200,188,433,1,360
top,School,667,0101,Gwinnett County,Lakeside High School,ALL Students,100.00
freq,442,26,14,26,2,442,18


In [ ]:
data

,Reporting_Level,System_ID,School_ID,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,School,601,0103,Appling County,Appling County High School,ALL Students,93.18
1,School,601,0103,Appling County,Appling County High School,American Indian/Alaskan,Too Few Students
2,School,601,0103,Appling County,Appling County High School,Asian/Pacific Islander,Too Few Students
3,School,601,0103,Appling County,Appling County High School,Black,84.21
4,School,601,0103,Appling County,Appling County High School,Economically Disadvantaged,93.18
...,...,...,...,...,...,...,...
6678,State,ALL,ALL,All Systems,All Schools,English Learners,68.49
6679,State,ALL,ALL,All Systems,All Schools,Hispanic,80.68
6680,State,ALL,ALL,All Systems,All Schools,Multi-Racial,87.08
6681,State,ALL,ALL,All Systems,All Schools,Students With Disability,74.01


In [ ]:
# !pip install transformers

In [ ]:
# Defined the tapas model and the pipelinein which we are going to use it.
from transformers import pipeline
tqa = pipeline(task="table-question-answering", 
               model="google/tapas-base-finetuned-wtq")

ImportError: ignored

In [ ]:
!pip install torch-scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data_soted_new

,System_Name,School_Name,Reporting_Label,Graduation_Rate
0,DeKalb County,Arabia Mountain High School - Academy of Engin...,ALL Students,99.67
1,Clayton County,Martha Ellen Stilwell School for the Performin...,ALL Students,99.28
2,Columbia County,Greenbrier High School,ALL Students,99.21
3,Atlanta Public Schools,Charles Drew Charter School JA/SA,ALL Students,99.10
4,Savannah-Chatham County,Savannah Arts Academy,ALL Students,99.10
...,...,...,...,...
437,Cobb County,Devereux Ackerman Academy,ALL Students,0.00
438,Savannah-Chatham County,UHS of Savannah Coastal Harbor Treatment Center,ALL Students,0.00
439,Taylor County,Georgia Center,ALL Students,0.00
440,Gwinnett County,Gwinnett Intervention Education Center (GIVE) ...,ALL Students,0.00


In [ ]:
data_new = data[(data['Reporting_Level']=='School') & (data['Reporting_Label']=='ALL Students')]
data_new1 = data_new[data_new['Graduation_Rate']!='Too Few Students']
data_sorted = data_new1.sort_values(by='Graduation_Rate',ascending = False)
data_soted_new = data_sorted[data_sorted['Graduation_Rate']!='No Data']
data_soted_new = data_soted_new.drop('Reporting_Level',axis=1)
data_soted_new.describe()
data_soted_new.drop(['System_ID','School_ID'],axis=1,inplace = True)
data_soted_new = data_soted_new.reset_index(drop='True')
all_new_data = pd.DataFrame(data_soted_new.to_dict('records'))

all_new_data.rename(column = {'System_Name':'system name',
                              'School_Name':'school name',
                              'Reporting_Label':'reporting',
                              'Graduation_Rate':'graduation'},inplace= True)

TypeError: ignored

In [ ]:
pos = range(1,443)
data_soted_new['Pos']=pos
data_soted_new.head()

,System_Name,School_Name,Reporting_Label,Graduation_Rate,Pos
0,DeKalb County,Arabia Mountain High School - Academy of Engin...,ALL Students,99.67,1
1,Clayton County,Martha Ellen Stilwell School for the Performin...,ALL Students,99.28,2
2,Columbia County,Greenbrier High School,ALL Students,99.21,3
3,Atlanta Public Schools,Charles Drew Charter School JA/SA,ALL Students,99.10,4
4,Savannah-Chatham County,Savannah Arts Academy,ALL Students,99.10,5


In [ ]:
data_soted_new['Pos'] = data_soted_new['Pos'].astype('str')

In [ ]:
qureies = ["what is the School_Name with highest Graduation_Rate"]
answer_list = tqa(table=data,
                  query=qureies,
                  )

In [ ]:
answer_list

{'answer': 'System',
 'coordinates': [(13, 0)],
 'cells': ['System'],
 'aggregator': 'NONE'}

In [ ]:
for answer in answer_list:
    print(answer["answer"])

1992-2007
Sachin Tendulkar
SUM > 18426, 14234, 13704, 13430, 12650, 11867, 11739, 11579, 11363, 10889
Sachin Tendulkar, Virat Kohli, Saurav Ganguly, Rahul Dravid


In [ ]:
answer

{'answer': 'Sachin Tendulkar, Virat Kohli, Saurav Ganguly, Rahul Dravid',
 'coordinates': [(0, 1), (5, 1), (8, 1), (9, 1)],
 'cells': ['Sachin Tendulkar',
  'Virat Kohli',
  'Saurav Ganguly',
  'Rahul Dravid'],
 'aggregator': 'NONE'}

### Tapas Modelling through the tokenizers

In [ ]:
!pip install torch-scatter transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 43.2 MB/s 
     |████████████████████████████████| 120 kB 64.4 MB/s 
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=0aefbb3925029d539b1ef67b06222f3d62e647aab71e0cfb3845efdfffbf0d7d
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


In [ ]:
from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline
import pandas as pd

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
graduate_data = pd.read_csv("/content/gdrive/MyDrive/graduate_rate.csv")
graduate_data = graduate_data.astype(str)

Mounted at /content/gdrive


In [ ]:
graduate_data= graduate_data.astype(str)

In [ ]:
# Load model & tokenizer
model = 'google/tapas-large-finetuned-wtq'
model1= 'microsoft/tapex-large-finetuned-wtq'
model2 = 'google/tapas-large-finetuned-sqa'
tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(model)
tapas_tokenizer = AutoTokenizer.from_pretrained(model)

# Initializing pipeline
tqa = pipeline('table-question-answering',
               model=tapas_model,
               tokenizer=tapas_tokenizer)

def questionanswering(query,data):
    response = tqa({'table': data,'query':query})
    answer = response['cells']
    print(answer)

In [ ]:
prediction = questionanswering('What is the School_Name with highest Graduataion_Rate',
                graduate_data[(graduate_data['Reporting_Level']=='School') & (graduate_data['Reporting_Label']=='ALL Students')].reset_index(drop=True).drop(['Reporting_Level','Reporting_Label'],axis=1).iloc[:310,:]
                )
#310 is the breaking point

[]


In [ ]:
graduate_data[(graduate_data['Reporting_Level']=='School') & (graduate_data['Reporting_Label']=='ALL Students')].reset_index(drop=True).drop(['Reporting_Level','Reporting_Label'],axis=1).iloc[:310,:]


,System_ID,School_ID,System_Name,School_Name,Graduation_Rate
0,601,0103,Appling County,Appling County High School,93.18
1,602,0103,Atkinson County,Atkinson County High School,94.35
2,603,0302,Bacon County,Bacon County High School,94.44
3,604,0105,Baker County,Baker County K12 School,Too Few Students
4,605,0189,Baldwin County,Baldwin High School,88.07
...,...,...,...,...,...
305,699,4050,Meriwether County,Manchester High School,76.79
306,700,0210,Miller County,Miller County High School,81.43
307,701,0282,Mitchell County,Mitchell County High School,87.21
308,702,0184,Monroe County,Mary Persons High School,92.11


Possible Reason for the Code Failure

1.   The vector size that the data going to create is not matching with the required model( most probably this is the reason, because when I am trying to feed less data it is ginv g me the response)
2.   The data type is not right




Next Steps:


*   will have to deifne the maximum size of tokens in tokenizer
*   have to tweak some of the parameters in model
*   Have to try different model of tapas(There are many different models that is included in the tapas library) to check for the accuracy



## Pipeline

In [1]:
!pip install torch-scatter transformers
!pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 24.2 MB/s 
     |████████████████████████████████| 6.6 MB 37.3 MB/s 
     |████████████████████████████████| 120 kB 45.6 MB/s 
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=eadb8e5509f77852ff79b29e8f59ac70884c5c228e04904abd1ceea3b3d6ba78
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline, TapexTokenizer, BartForConditionalGeneration
import pandas as pd
from google.colab import drive

In [3]:
drive.mount('/content/gdrive')
graduate_data = pd.read_csv("/content/gdrive/MyDrive/graduate_rate.csv")
graduate_data = graduate_data.astype(str)

Mounted at /content/gdrive


In [4]:

dict1 = {
         'Tapas Large Wtq' : { "model_name":'google/tapas-large-finetuned-wtq'},
         'Tapas Base' : {"model_name":'google/tapas-base'},
         'Tapas base WTQ' : {"model_name":"google/tapas-base-finetuned-wtq"},
         'Tapas  Base TabFact' : {"model_name": "google/tapas-base-finetuned-tabfact"},
         'Tapas large Masked' : {"model_name" : 'google/tapas-large-masklm'},
         'Tapas base Masked' : {"model_name" : 'google/tapas-base-masklm'},
         'Tapex Large WTQ' : {"model_name" : 'microsoft/tapex-large-finetuned-wtq'},
         'Tapex t5 Adapt' : {"model_name" : 'microsoft/tapex-large-sql-execution'},
         'Tapex Base WTQ' : {"model_name" : 'microsoft/tapex-base-finetuned-wtq'},
         'Tapex Base wikisql' : {"model_name" : 'microsoft/tapex-base-finetuned-wikisql'}
         }

for key,val in dict1.items():
  if 'Tapas' in key:
    tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(val["model_name"])
    tapas_tokenizer = AutoTokenizer.from_pretrained(val["model_name"])
    val['tokenizer'] = tapas_tokenizer
    val['model'] = tapas_model
  else:
    tapex_tokenizer = TapexTokenizer.from_pretrained(val["model_name"]) #we can use medium as well
    tapex_model = BartForConditionalGeneration.from_pretrained(val["model_name"])
    val['tokenizer'] = tapex_tokenizer
    val['model'] = tapex_model

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'output_bias', 'output_weights', 'column_output_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of the model checkpoint at google/tapas-base-finetuned-tabfact were not used when initializing TapasForQuestionAnswering: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TapasForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TapasForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base-finetuned-tabfact and are newly initialized: ['column_output_weights', 'output_bias', 'output_weights', 'column_output_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

Some weights of the model checkpoint at google/tapas-large-masklm were not used when initializing TapasForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TapasForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TapasForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-large-masklm and are n

Some weights of the model checkpoint at google/tapas-base-masklm were not used when initializing TapasForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TapasForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TapasForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base-masklm and are new

In [ ]:
# for key in list(dict1.keys())[-3:]:
#   val = dict1[key]
#   tapex_tokenizer = TapexTokenizer.from_pretrained(val["model_name"]) #we can use medium as well
#   tapex_model = BartForConditionalGeneration.from_pretrained(val["model_name"])
#   val['tokenizer'] = tapex_tokenizer
#   val['model'] = tapex_model
# def questionanswering(query,data):
#     tqa = pipeline('table-question-answering',
#                model=dict1['Tapas Large Wtq']['model'],
#                tokenizer=dict1['Tapas Large Wtq']['tokenizer'])
    
#     response = tqa({'table': data,'query':query})
#     print(response)



In [5]:

## Data Cleaning Function
def data_clean(data):
  data_new = data[(data['Reporting_Level']=='School') &
                  (data['Reporting_Label']=='ALL Students') &
                  (data['Graduation_Rate']!='Too Few Students') &
                  (data['Graduation_Rate']!='No Data')]
  
  data_sorted = data_new.sort_values(by='Graduation_Rate',
                                     ascending = False)
  
  data_sorted.drop(['System_ID','School_ID','Reporting_Level'],
                      axis=1,
                      inplace = True)
  data_soted_new = data_sorted.reset_index(drop='True')
  return data_soted_new

cleaned_data = data_clean(graduate_data)


In [27]:
list_ofquestion = [
            {"question":"what is the highest score for Graduation_Rate for the System_Name as DeKalb County",
             "answer":"99.67"},
            {"question":"How many School_Name has Graduation_Rate above to 90",
             "answer":"262"},
            {"question":"What is the Reporting_Label for School_name as Charles Drew Charter School JA/SA",
             "answer":"All Students"},
            {"question":"What is the count of System_Name for the Graduation_Rate above to 80",
             "answer":"380"},
            {"question":"how many system_name are their for Clayton County",
             "answer":"12"},
            {"question":"what is the maximum Graduation_Rate for School_name as Savannah Arts Academy",
             "answer":"99.10"}
            ]

def askQuestion(query,table,tokenizer,model):
    encoding = tokenizer(table=table, query=query, return_tensors="pt", max_length =1024, truncation=True)
    outputs = model.generate(**encoding)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

for key,val in dict1.items():
  ini_model = val['model']
  init_token = val['tokenizer']

  if 'Tapas' in key:
    tqa = pipeline('table-question-answering',
          model=ini_model,
          tokenizer=init_token)
    for question in list_ofquestion:
      try:
        response = tqa({'table': cleaned_data.iloc[:100,:],
                        'query':question['question']})
      except Exception as exp:
        print(exp)
      question[key] = response['answer']
  else:
    for question in list_ofquestion:
      response = askQuestion(query= question['question'],
                            table = cleaned_data.iloc[:100,:],
                            tokenizer = init_token,
                            model = ini_model )
      question[key] = response


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 1024 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [28]:
list_ofquestion

[{'question': 'what is the highest score for Graduation_Rate for the System_Name as DeKalb County',
  'answer': '99.67',
  'Tapas Large Wtq': 'SUM > 99.67',
  'Tapas Base': '',
  'Tapas base WTQ': 'AVERAGE > 99.67',
  'Tapas  Base TabFact': 'ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, ALL Students, AL

In [29]:
results = pd.DataFrame(list_ofquestion)
results.to_csv('./gdrive/MyDrive/results.csv',index=False)